# Text to text and text to image with Amazon Nova

This hands-on section will guide you through how to use Nova text and image generation

## 🎯 What You'll Build

By the end of this workshop, you'll have:
- ✅ An understanding of how to generate text and images with Nova
- ✅ Examples to get you started

## Workshop Outline
1. Text generation
2. Built-in reasoning
3. Image generation with text rendering

## Text generation

During this section, you will send requests to generate text with word count requirements and structured output.

In [ ]:
# Install required packages (it can take several minutes)
!pip install -r requirements.txt --quiet

Set variables and helper functions:

In [10]:
import boto3
from botocore.config import Config
import json

REGION_ID = "us-west-2"
MODEL_ID = "us.amazon.nova-2-omni-v1:0"

bedrock_runtime = boto3.client('bedrock-runtime', region_name=REGION_ID)


### Word count
You can control how long the response should be using natural language in the input prompt, in addition to the maxToken paramater. In the following example, you will send a request to generate a response in 10 or less words.

In [ ]:
request = {
    "modelId": MODEL_ID,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "In *less than 10 words**, describe what a Hello World program is."
                }
            ],
        }
    ],
    "inferenceConfig": {"temperature": 0.5, "maxTokens": 512},
}
response = bedrock_runtime.converse(**request)
response_content_list = response["output"]["message"]["content"]

# Extract text content block
text_content = next(
    (item for item in response_content_list if "text" in item),
    None,
)

if text_content:
    print("== Text Output ==")
    print(text_content["text"])


### Use output formats
Nova can generate outputs in specific formats such as JSON. Having structured output provides development teams with a better experience because less code is needed to parse the output of the model. The following code generates a list of grocery products using a JSON schema, with variable types and required fields.

In [ ]:
system_prompt = """
Respond using this schema:
{
  "type": "object",
  "properties": {
    "item_id": {
      "type": "integer"
    },
    "name": {
      "type": "string"
    },
    "description": {
      "type": "string"
    }
  },
  "required": ["item_id", "name", "description"]
}

"""
request = {
    "modelId": MODEL_ID,
    "system": [{"text": system_prompt}],
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "Return multiple every day grocery items"
                }
            ],
        }
    ],
    "inferenceConfig": {"temperature": 0.5, "maxTokens": 1024},
}
response = bedrock_runtime.converse(**request)
response_content_list = response["output"]["message"]["content"]

# Extract text content block
text_content = next(
    (item for item in response_content_list if "text" in item),
    None,
)

if text_content:
    # Validate JSON output
    json_output = json.loads(text_content["text"].replace("```","").replace("json", ""))
    print("== JSON Output ==")
    print(json.dumps(json_output, indent=2))


## Built-in Reasoning
Instead of asking Nova to think step by step, you can enable a parameter to enable the reasoning config, with a high effort. Having this as a setting simplifies the prompt and can be validated using the reasoningContent section in the response

In [ ]:
system_prompt = """
Respond using this schema:
{
  "type": "object",
  "properties": {
    "result": {
      "type": "string"
    }
  },
  "required": ["result"]
}

"""
text_prompt="""
Explain why plants need sunlight, water, and carbon dioxide to survive.
"""
request = {
    "modelId": MODEL_ID,
    "system": [{"text": system_prompt}],
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": text_prompt
                }
            ],
        }
    ],
    "additionalModelRequestFields":{'reasoningConfig': {'type': 'enabled', 'maxReasoningEffort': 'high'}}
}
response = bedrock_runtime.converse(**request)
response_content_list = response["output"]["message"]["content"]
reasoning_content = next(
    (item for item in response_content_list if "reasoningContent" in item),
    None,
)
print("== Reasoning ==")
print(json.dumps(reasoning_content,indent=2)) 
# Extract text content block
text_content = next(
    (item for item in response_content_list if "text" in item),
    None,
)

if text_content:
    # Validate JSON output
    json_output = json.loads(text_content["text"].replace("```","").replace("json", ""))
    print("== Result ==") 
    print(json_output['result'])


## Image generation with text rendering
In this section, you will send a request to Nova to generate an image of a new car model. The prompt goes into the details that the model needs to consider while generating the image and what text should be rendered

In [ ]:
import json
import timeit
from botocore.exceptions import ClientError
from IPython.display import Image, display
system_prompt = "You are a professional automotive CGI designer. All the images generated have to be in 1:1 ratio."
prompt = """
You must create a high-resolution photorealistic render based on the following instructions without deviating from the details:
1. The subject is an All-New SUV, 'Sensory Captain Chairs' grade, white exterior. 
2. The scene should place the vehicle on a scenic beach backdrop during golden hour with realistic reflections and lighting. 
3. Overlay the exact text “The Pinnacle of Luxury Experience” in an elegant, premium serif font at the lower third of the image. 
4. Maintain a cinematic composition, showcasing the full car exterior in a three-quarter front view.
Now generate the image of the car
"""
request = {
    "modelId": MODEL_ID,
    "system": [{"text": system_prompt}],
    "messages": [
        {
            "role": "user",
            "content": [
                 {"text": prompt} 
            ],
        }
    ],
    "inferenceConfig": {"temperature": 0.5, "maxTokens": 512},
}

# Start the request
start = timeit.default_timer()
response = bedrock_runtime.converse(**request)
elapsed = timeit.default_timer() - start
print(f"Request took {elapsed:.2f} seconds")


Because Nova has multiple output modalities, it can include images and text in the response. The following code checks if an image is returned by the model, gets the bytes and displays it

In [ ]:

try:
    response_content_list = response["output"]["message"]["content"]

    # Extract image content block
    image_content = next(
        (item for item in response_content_list if "image" in item),
        None,
    )

    if image_content:
        print("== Image Output ==")
        image_bytes = image_content["image"]["source"]["bytes"]
        generated_image = Image(data=image_bytes)
        display(generated_image)

except ClientError as err:
    print("Error occurred:")
    print(err)
    if hasattr(err, "response"):
        print(json.dumps(err.response, indent=2))

## 🎯 Workshop Summary

Congratulations! You've successfully completed the text to text and text to image workshop with Amazon Nova. Great job! 🎉